In [1]:
%run imports_and_functions.ipynb

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Installing YelpAPI

In [18]:
!pip install yelpapi

# Connection with yelp api managed through github sourced yelpapi, author gfairchild, permitted with
# display of copyright notice below.
"""
    Copyright (c) 2013, Los Alamos National Security, LLC
    All rights reserved.
    Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
    following conditions are met:
    * Redistributions of source code must retain the above copyright notice, this list of conditions and the following
      disclaimer.
    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the
      following disclaimer in the documentation and/or other materials provided with the distribution.
    * Neither the name of Los Alamos National Security, LLC nor the names of its contributors may be used to endorse or
      promote products derived from this software without specific prior written permission.
    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,
    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
    SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
    SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
    WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
    OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

'\n    Copyright (c) 2013, Los Alamos National Security, LLC\n    All rights reserved.\n    Redistribution and use in source and binary forms, with or without modification, are permitted provided that the\n    following conditions are met:\n    * Redistributions of source code must retain the above copyright notice, this list of conditions and the following\n      disclaimer.\n    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the\n      following disclaimer in the documentation and/or other materials provided with the distribution.\n    * Neither the name of Los Alamos National Security, LLC nor the names of its contributors may be used to endorse or\n      promote products derived from this software without specific prior written permission.\n    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,\n    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHAN

## Global Variables

In [2]:
from yelpapi import YelpAPI

API_KEY1 = "G1-37Mu0IHzYjviLF5gdpXJ_GdNZhKBCVuVLCfwJ5oRLivMCKpWrQMYTurfGuihavP0K7gOiLpH-0Ze75L96RjthUEoMukghlMqexUaziQ4agkeu6MfdtW5enpgvXHYx"

yelp_api = YelpAPI(API_KEY1)

In [ ]:
# Expected useful business subcategories:
# Restaurants (restaurants, All)
# Food (food, All)
# Shopping (shopping, All)
# Arts & Entertainment (arts, All)
#     -> Art Galleries (galleries, All)
#     -> Country Clubs (countryclubs
#     -> Art Museums (artmuseums, 
#     -> Performing Arts (theater, All)
# Nightlife (nightlife, All)
# Beauty & Spas (beautysvc, All)
# Active Life 
#     -> Fitness & Instruction (fitness, All)
#     -> Golf

# valet, casual/formal attire

## Extracting Area Data from Yelp

### Storing data for one Yelp Search

In [3]:
class SearchData(object) :
    
    # Search parameters must always be given a location, for this project always a street address
    # For one location and one main_category of business
    
    def __init__(self, name, main_category, location=None, coords=(90,90), 
                 limit = 50, offset = 0, radius=search_radius, df=None) :
        self.name = name
        self.main_category = main_category
        self.coords = coords
        self.location = location # assume input of either text location and if not lat/long coords
        self.limit = limit
        self.offset = offset
        self.radius = radius  # radius (meters) to search within, yelp sometimes grabs outside the radius
        if df == None :
            self.df = pd.DataFrame(columns = features)
        else : self.df = df
        self.sort_by = 'distance'

    # other optional parameters
    #     term    # user inputted search term
    #     sort_by # default is best_match, other options include distance and review_count but these may 
    # limit max search results returned to only 40, per yelp documentation

    def printparams(self) :
        print('location: ', self.location, ', coords: ', self.coords, ', offset: ', self.offset, 
        ', radius: ', self.radius, ', main_category: ', self.main_category)
        print('Dataframe size:', self.df.shape)
    
    def add50rows(self) :
        latlong = None
        if self.location == None :  
            lat, long = self.coords
            latlong = str(lat) + ', ' + str(long)
            location = latlong
        else : location = self.location
        search_results = yelp_api.search_query(location=location, 
                                               limit=self.limit, offset=self.offset,
                                               categories=self.main_category, sort_by=self.sort_by)
        prev_count = self.df.shape[0]
        max_distance = 0
        for business in search_results['businesses'] :
            self.df = self.df.append({tupl[0] : tupl[1] for tupl in business.items() if tupl[0] in features},
                                    ignore_index=True, sort=True)
            if  business['distance'] > max_distance : max_distance = business['distance']
#         print(self.df.shape[0] - prev_count, 'rows added to', self.name, ' for ', self.main_category)
        return max_distance
    
    def addyelpdata(self) :
        end = False
        max_distance = 0
        while (self.offset < 1000) & (not end) :
            try :
                max_distance = self.add50rows()
                if max_distance > self.radius : end = True
                self.offset += self.limit
            except : # usually this means at end of yelp search results or hit 1000 result hard limit
                end = True
        
        self.df['latitude'] = self.df['coordinates'].map((lambda x : x['latitude']))
        self.df['longitude'] = self.df['coordinates'].map((lambda x : x['longitude']))
        self.df['main_category'] = self.main_category
        print(self.main_category, ' done, size: ', self.df.shape, ' max_dist = ', max_distance)

### Functions for One City Searches for Every Business Category 

In [4]:
class AreaData(object) :

    def __init__(self, name, coords=(90,90), location=None, radius=search_radius,
                df=pd.DataFrame()) :
        self.name = name
        self.coords = coords
        self.location = location
        self.radius = radius  # radius (meters) to search within, yelp sometimes grabs outside the radius
        self.df = df
        self.searches = []

    def extract_yelp_businesses(self, location=None, coords=(90,90)) :
        for category in main_alias :
            search = SearchData('boston1', category, location=location, coords=coords, radius=self.radius)
            search.addyelpdata()
            self.df = self.df.append(search.df, ignore_index = True, sort=True)
            self.searches.append(search)
        return self.df

In [26]:
def yelp_extra_square_search(main_category, lat, long, df=pd.DataFrame()) :
    lat1000m = -115.161605-(-115.172713)
    scale = 5
    corner1 = (lat-lat1000m*scale, long-lat1000m*scale)
    corner2 = (lat+lat1000m*scale, long-lat1000m*scale)
    corner3 = (lat+lat1000m*scale, long+lat1000m*scale)
    corner4 = (lat-lat1000m*scale, long+lat1000m*scale)
    top = lat+lat1000m*scale, long
    bottom = lat-lat1000m*scale, long
    left = lat, long-lat1000m*scale
    right = lat, long+lat1000m*scale
    coordlist = [corner1, left, corner2, top, corner3, right, corner4, bottom]
    for coord in coordlist :
        search = SearchData('subsets', main_category, coords=coord, radius=1000*scale*0.7)
        search.addyelpdata()
        df = df.append(search.df, ignore_index = True, sort=True)
    display = df.shape
    dropmask = df[['alias','latitude','longitude','review_count']].duplicated(keep='first')
    df = df[~dropmask]
    print(display, ' dropped to ', df.shape)
    return df

## Extracting Boston Businesses within 4500 meter Radius

In [9]:
maxradius = 4500
testlat = 42.349397
testlong = -71.092480 # Boston, MA

In [7]:
boston1 = AreaData('Boston1', coords=(testlat, testlong), radius=maxradius)
df = boston1.extract_yelp_businesses(coords=(testlat, testlong))
df.shape

active  done, size:  (800, 12)  max_dist =  5067.800938992921
arts  done, size:  (600, 12)  max_dist =  6190.962542694475
automotive  done, size:  (500, 12)  max_dist =  5266.645117196608
beautysvc  done, size:  (1000, 12)  max_dist =  4512.937708552031
education  done, size:  (450, 12)  max_dist =  5130.022659200853
eventservices  done, size:  (800, 12)  max_dist =  5058.790089764473
financialservices  done, size:  (400, 12)  max_dist =  5268.37530991864
food  done, size:  (1000, 12)  max_dist =  3339.691995487489
health  done, size:  (1000, 12)  max_dist =  3035.3559560361236
homeservices  done, size:  (1000, 12)  max_dist =  3078.7656519261473
hotelstravel  done, size:  (700, 12)  max_dist =  5329.777458536324
localservices  done, size:  (1000, 12)  max_dist =  4309.098729159692
massmedia  done, size:  (88, 12)  max_dist =  11352.053314398836
nightlife  done, size:  (750, 12)  max_dist =  5266.345251586332
pets  done, size:  (200, 12)  max_dist =  5746.45478686026
professional  done

(14538, 12)

In [9]:
df[~df['price'].isnull()]

,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count
3,joint-ventures-physical-therapy-and-fitness-bo...,NaN,"[{'alias': 'physicaltherapy', 'title': 'Physic...","{'latitude': 42.34939, 'longitude': -71.0959}",250.918151,42.349390,-71.095900,active,Joint Ventures Physical Therapy and Fitness,$$,4.5,54
5,lucky-strike-boston,NaN,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...","{'latitude': 42.3474085, 'longitude': -71.0943...",255.469235,42.347409,-71.094304,active,Lucky Strike,$$,2.5,458
11,lululemon-athletica-boston-4,NaN,"[{'alias': 'sportswear', 'title': 'Sports Wear...","{'latitude': 42.3487243652344, 'longitude': -7...",512.167399,42.348724,-71.086693,active,lululemon athletica,$$$,4.0,24
16,kings-dining-and-entertainment-boston-back-bay...,NaN,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...","{'latitude': 42.34706, 'longitude': -71.08565}",646.675006,42.347060,-71.085650,active,Kings Dining & Entertainment - Boston Back Bay,$$,3.0,478
25,massage-for-yoga-boston,NaN,"[{'alias': 'massage', 'title': 'Massage'}, {'a...","{'latitude': 42.34997, 'longitude': -71.08255}",851.882216,42.349970,-71.082550,active,Massage for Yoga,$$$,5.0,1
82,cambridge-bicycle-cambridge,NaN,"[{'alias': 'bikerentals', 'title': 'Bike Renta...","{'latitude': 42.3617599, 'longitude': -71.09715}",1431.274241,42.361760,-71.097150,active,Cambridge Bicycle,$$,3.5,192
86,equinox-dartmouth-boston,NaN,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...","{'latitude': 42.34683, 'longitude': -71.07542}",1463.009491,42.346830,-71.075420,active,Equinox Dartmouth,$$$$,3.5,129
128,saldare-body-therapy-boston-2,NaN,"[{'alias': 'massage', 'title': 'Massage'}, {'a...","{'latitude': 42.3446822, 'longitude': -71.0723...",1765.935107,42.344682,-71.072354,active,Saldare Body Therapy,$$,5.0,8
151,exhale-back-bay-boston,NaN,"[{'alias': 'spas', 'title': 'Day Spas'}, {'ali...","{'latitude': 42.3517198, 'longitude': -71.0704...",1872.368986,42.351720,-71.070422,active,Exhale Back Bay,$$$,3.5,290
180,get-tight-sport-fishing-boston-2,NaN,"[{'alias': 'huntingfishingsupplies', 'title': ...","{'latitude': 42.3455626, 'longitude': -71.0693...",1982.954232,42.345563,-71.069308,active,Get Tight Sport Fishing,$$,5.0,2


In [10]:
df[~df['attributes'].isnull()]

,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count


In [15]:
# Business categories that have more than 1000 limit: food, health, homeservices,
# localservices, restaurants, shopping

bostonextra1 = yelp_extra_square_search('food', testlat, testlong)

food  done, size:  (150, 11)  max_dist =  2900.798284376997
food  done, size:  (300, 11)  max_dist =  2957.124693293625
food  done, size:  (300, 11)  max_dist =  2729.610847750787
food  done, size:  (300, 11)  max_dist =  2785.325868940938
food  done, size:  (400, 11)  max_dist =  2730.3464952004174
food  done, size:  (700, 11)  max_dist =  2722.6794775571097
food  done, size:  (200, 11)  max_dist =  2783.8317171611393
food  done, size:  (12, 11)  max_dist =  0
(2362, 11)  dropped to  (1742, 11)


In [8]:
bostonextra2 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (200, 11)  max_dist =  2888.2602648007764
restaurants  done, size:  (400, 11)  max_dist =  2735.791127010438
restaurants  done, size:  (450, 11)  max_dist =  2756.019300892678
restaurants  done, size:  (400, 11)  max_dist =  2812.2875926701295
restaurants  done, size:  (600, 11)  max_dist =  2726.6328749153663
restaurants  done, size:  (1000, 11)  max_dist =  2372.8635562376203
restaurants  done, size:  (300, 11)  max_dist =  2867.1000827192047
restaurants  done, size:  (26, 11)  max_dist =  0
(3376, 11)  dropped to  (2562, 11)


In [12]:
bostonextra3 = yelp_extra_square_search('shopping', testlat, testlong)

shopping  done, size:  (150, 11)  max_dist =  2750.594420685027
shopping  done, size:  (300, 11)  max_dist =  2908.290893110274
shopping  done, size:  (400, 11)  max_dist =  2828.237204112633
shopping  done, size:  (350, 11)  max_dist =  2983.9885610271563
shopping  done, size:  (400, 11)  max_dist =  2740.6844789394513
shopping  done, size:  (950, 11)  max_dist =  2754.0752316587887
shopping  done, size:  (200, 11)  max_dist =  3017.7658776437383
shopping  done, size:  (13, 11)  max_dist =  0
(2763, 11)  dropped to  (2125, 11)


In [13]:
bostonextra4 = yelp_extra_square_search('homeservices', testlat, testlong)

homeservices  done, size:  (200, 11)  max_dist =  2780.4170700914556
homeservices  done, size:  (450, 11)  max_dist =  2936.0544226237685
homeservices  done, size:  (400, 11)  max_dist =  2717.0403184610627
homeservices  done, size:  (350, 11)  max_dist =  2712.730636424124
homeservices  done, size:  (400, 11)  max_dist =  2723.238480609732
homeservices  done, size:  (750, 11)  max_dist =  2785.6308606444377
homeservices  done, size:  (200, 11)  max_dist =  2753.512440005919
homeservices  done, size:  (15, 10)  max_dist =  4341.428184729574
(2765, 11)  dropped to  (2089, 11)


In [10]:
bostonextra5 = yelp_extra_square_search('health', testlat, testlong)

health  done, size:  (150, 12)  max_dist =  2907.939941121806
health  done, size:  (550, 12)  max_dist =  2903.3879838147877
health  done, size:  (350, 12)  max_dist =  2914.5399172479333
health  done, size:  (250, 12)  max_dist =  2888.796689846484
health  done, size:  (150, 12)  max_dist =  3352.255745067271
health  done, size:  (650, 12)  max_dist =  3188.023635627211
health  done, size:  (100, 12)  max_dist =  3791.284917601141
health  done, size:  (5, 12)  max_dist =  0
(2205, 12)  dropped to  (1934, 12)


In [11]:
bostonextra5

,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count
0,armenian-nursing-and-rehab-center-jamaica-plain,NaN,"[{'alias': 'rehabilitation_center', 'title': '...","{'latitude': 42.31022, 'longitude': -71.13187}",817.171647,42.310220,-71.131870,health,Armenian Nursing & Rehab Center,NaN,5.0,1
1,springhouse-senior-living-boston,NaN,"[{'alias': 'retirement_homes', 'title': 'Retir...","{'latitude': 42.299906, 'longitude': -71.131641}",973.322883,42.299906,-71.131641,health,Springhouse Senior Living,NaN,1.0,1
2,james-adam-greenberg-md-boston,NaN,"[{'alias': 'obgyn', 'title': 'Obstetricians & ...","{'latitude': 42.301649, 'longitude': -71.1287817}",984.139443,42.301649,-71.128782,health,"James Adam Greenberg, MD",NaN,5.0,2
3,eva-m-balash-md-jamaica-plain,NaN,"[{'alias': 'dermatology', 'title': 'Dermatolog...","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,"Eva M Balash, MD",NaN,3.5,7
4,brigham-and-womens-faulkner-hospital-jamaica-p...,NaN,"[{'alias': 'hospitals', 'title': 'Hospitals'}]","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,Brigham and Women's Faulkner Hospital,NaN,3.0,44
5,faulkner-sagoff-breast-imaging-and-diagnostic-...,NaN,"[{'alias': 'diagnosticimaging', 'title': 'Diag...","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,Faulkner Sagoff Breast Imaging & Diagnostic Ce...,NaN,2.0,8
6,matthew-p-butler-dpm-jamaica-plain,NaN,"[{'alias': 'podiatrists', 'title': 'Podiatrist...","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,"Matthew P Butler, DPM",NaN,5.0,3
7,boston-family-dermatology-jamaica-plain,NaN,"[{'alias': 'dermatology', 'title': 'Dermatolog...","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,Boston Family Dermatology,NaN,4.0,1
8,duggan-margaret-md-jamaica-plain,NaN,"[{'alias': 'hospitals', 'title': 'Hospitals'}]","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,Duggan Margaret MD,NaN,5.0,1
9,hartigan-joseph-a-dr-podiatrst-jamaica-plain,NaN,"[{'alias': 'podiatrists', 'title': 'Podiatrist...","{'latitude': 42.3003492, 'longitude': -71.1278...",984.520798,42.300349,-71.127867,health,Hartigan Joseph A Dr Podiatrst,NaN,2.0,1


In [16]:
bostonextra6 = yelp_extra_square_search('localservices', testlat, testlong)

(13938, 11)

In [19]:
dfboston = df.append([bostonextra1,bostonextra2,bostonextra3,bostonextra4,bostonextra5], 
                     ignore_index = True, sort=True)
dfboston.shape

(24243, 11)

In [20]:
dropmask = dfboston[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dfboston = dfboston[~dropmask]
dfboston.shape

(16713, 11)

In [21]:
dfboston.to_csv('BostonBusinesses.csv', sep=';', compression='gzip')

### Extracting All Miami Business Categories Within 6000m Radius

In [23]:
maxradius = 6000
testlat = 25.786686
testlong = -80.237440 # Miami, FL

miami1 = AreaData('Miami1', coords=(testlat, testlong), radius=maxradius)
df2 = miami1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (350, 11)  max_dist =  9669.235738091327
arts  done, size:  (300, 11)  max_dist =  6017.16371897082
automotive  done, size:  (500, 11)  max_dist =  6229.019916764452
beautysvc  done, size:  (750, 11)  max_dist =  6060.315942143552
education  done, size:  (200, 11)  max_dist =  22210.16377701274
eventservices  done, size:  (500, 11)  max_dist =  7226.220173126322
financialservices  done, size:  (250, 11)  max_dist =  15387.271704285784
food  done, size:  (750, 11)  max_dist =  6269.224020260538
health  done, size:  (800, 11)  max_dist =  6204.862321126451
homeservices  done, size:  (800, 11)  max_dist =  6204.862321126451
hotelstravel  done, size:  (500, 11)  max_dist =  9847.509876867727
localservices  done, size:  (600, 11)  max_dist =  6181.552303564595
massmedia  done, size:  (19, 10)  max_dist =  0
nightlife  done, size:  (350, 11)  max_dist =  42318.61367409768
pets  done, size:  (100, 10)  max_dist =  6149.7572511161125
professional  done, size:  (650, 11)  m

In [27]:
# restaurants and shopping were over 1000 max results
miamiextra1 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (250, 11)  max_dist =  3722.8468324712117
restaurants  done, size:  (300, 11)  max_dist =  3821.0976811840665
restaurants  done, size:  (250, 11)  max_dist =  3543.7625064082144
restaurants  done, size:  (150, 11)  max_dist =  3939.9332417495334
restaurants  done, size:  (200, 11)  max_dist =  3701.8826347198433
restaurants  done, size:  (800, 11)  max_dist =  3650.6625070513674
restaurants  done, size:  (500, 11)  max_dist =  3587.668213480422
restaurants  done, size:  (350, 11)  max_dist =  3939.2833868553544
(2800, 11)  dropped to  (2167, 11)


In [28]:
miamiextra2 = yelp_extra_square_search('shopping', testlat, testlong)

shopping  done, size:  (400, 11)  max_dist =  3683.4695169888196
shopping  done, size:  (250, 11)  max_dist =  3573.5062033388745
shopping  done, size:  (300, 11)  max_dist =  3704.1789850383884
shopping  done, size:  (150, 11)  max_dist =  4186.14573094513
shopping  done, size:  (250, 11)  max_dist =  3768.051749339579
shopping  done, size:  (700, 11)  max_dist =  3867.402304053047
shopping  done, size:  (350, 11)  max_dist =  3721.724592510844
shopping  done, size:  (400, 11)  max_dist =  4029.381291362038
(2800, 11)  dropped to  (2179, 11)


In [29]:
dfmiami = df2.append([miamiextra1,miamiextra2], 
                     ignore_index = True, sort=True)
dfmiami.shape

(14256, 11)

In [30]:
dropmask = dfmiami[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dfmiami = dfmiami[~dropmask]
dfmiami.shape

(10571, 11)

In [31]:
dfmiami.to_csv('MiamiBusinesses.csv', sep=';', compression='gzip')

### Extracting All Seattle Business Categories Within 6000m Radius

In [50]:
maxradius = 6000
testlat = 47.592989
testlong = -122.336087 # Seattle, WA
 
seattle1 = AreaData('Seattle1', coords=(testlat, testlong), radius=maxradius)
df3 = seattle1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (700, 11)  max_dist =  6348.705632562032
arts  done, size:  (600, 11)  max_dist =  6779.734890882188
automotive  done, size:  (450, 11)  max_dist =  6047.138833730911
beautysvc  done, size:  (900, 11)  max_dist =  6340.297581553811
education  done, size:  (300, 11)  max_dist =  6528.126962980084
eventservices  done, size:  (850, 11)  max_dist =  6301.796385755899
financialservices  done, size:  (300, 11)  max_dist =  6172.185182290553
food  done, size:  (1000, 11)  max_dist =  4039.289599313987
health  done, size:  (1000, 11)  max_dist =  3131.1984622701916
homeservices  done, size:  (1000, 11)  max_dist =  2973.6927702144862
hotelstravel  done, size:  (450, 11)  max_dist =  6041.253310263168
localservices  done, size:  (850, 11)  max_dist =  6300.357119277919
massmedia  done, size:  (73, 11)  max_dist =  7826.140514325211
nightlife  done, size:  (600, 11)  max_dist =  6416.628095466277
pets  done, size:  (200, 11)  max_dist =  6432.26298639708
professional  done, 

In [48]:
# restaurants, food, health, homeservices, realestate and shopping were over 1000 max results
seattleextra1 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (19, 11)  max_dist =  0
restaurants  done, size:  (0, 11)  max_dist =  0
restaurants  done, size:  (400, 11)  max_dist =  4553.464562734669
restaurants  done, size:  (950, 11)  max_dist =  4242.1729730117995
restaurants  done, size:  (300, 11)  max_dist =  4401.5013952679965
restaurants  done, size:  (37, 11)  max_dist =  0
restaurants  done, size:  (150, 11)  max_dist =  4396.3284505854235
restaurants  done, size:  (200, 11)  max_dist =  4270.550137845675
(2056, 11)  dropped to  (1389, 11)


In [51]:
seattleextra2 = yelp_extra_square_search('food', testlat, testlong)

food  done, size:  (100, 11)  max_dist =  3581.009364285407
food  done, size:  (100, 11)  max_dist =  3996.157956567343
food  done, size:  (300, 11)  max_dist =  3905.2925030047472
food  done, size:  (600, 11)  max_dist =  3611.453310062469
food  done, size:  (200, 11)  max_dist =  3879.244762721199
food  done, size:  (200, 11)  max_dist =  3597.2319147926655
food  done, size:  (150, 11)  max_dist =  3922.361018260247
food  done, size:  (200, 11)  max_dist =  3906.1119555170435
(1850, 11)  dropped to  (1341, 11)


In [52]:
seattleextra3 = yelp_extra_square_search('health', testlat, testlong)

health  done, size:  (200, 11)  max_dist =  4040.488886536056
health  done, size:  (100, 11)  max_dist =  3507.739053850811
health  done, size:  (400, 11)  max_dist =  3669.343865557041
health  done, size:  (850, 11)  max_dist =  3543.513922945822
health  done, size:  (350, 11)  max_dist =  3644.7579396716255
health  done, size:  (200, 11)  max_dist =  3567.9663799111245
health  done, size:  (100, 11)  max_dist =  4183.2320547178715
health  done, size:  (100, 11)  max_dist =  3744.231427020052
(2300, 11)  dropped to  (1712, 11)


In [53]:
seattleextra4 = yelp_extra_square_search('homeservices', testlat, testlong)

homeservices  done, size:  (250, 11)  max_dist =  3676.4788008608384
homeservices  done, size:  (200, 11)  max_dist =  3790.8492156243765
homeservices  done, size:  (450, 11)  max_dist =  3561.5855158670774
homeservices  done, size:  (1000, 11)  max_dist =  3478.3233478707693
homeservices  done, size:  (250, 11)  max_dist =  3659.2127908972207
homeservices  done, size:  (250, 11)  max_dist =  3559.4697739198814
homeservices  done, size:  (200, 11)  max_dist =  3689.172700672888
homeservices  done, size:  (300, 11)  max_dist =  3654.5894976722307
(2900, 11)  dropped to  (2202, 11)


In [54]:
seattleextra5 = yelp_extra_square_search('realestate', testlat, testlong)

realestate  done, size:  (150, 11)  max_dist =  5237.740979727044
realestate  done, size:  (100, 11)  max_dist =  3865.8391693602803
realestate  done, size:  (200, 11)  max_dist =  3662.758574069639
realestate  done, size:  (700, 11)  max_dist =  3557.862349039909
realestate  done, size:  (150, 11)  max_dist =  3645.4082909418703
realestate  done, size:  (100, 11)  max_dist =  3546.056726815232
realestate  done, size:  (50, 11)  max_dist =  5072.161054178609
realestate  done, size:  (50, 11)  max_dist =  3760.379429124018
(1500, 11)  dropped to  (1101, 11)


In [55]:
seattleextra6 = yelp_extra_square_search('shopping', testlat, testlong)

shopping  done, size:  (150, 11)  max_dist =  4624.65265159942
shopping  done, size:  (100, 11)  max_dist =  3956.584555877006
shopping  done, size:  (350, 11)  max_dist =  3678.930683251335
shopping  done, size:  (650, 11)  max_dist =  3571.19380299418
shopping  done, size:  (250, 11)  max_dist =  3685.5859981044123
shopping  done, size:  (150, 11)  max_dist =  3522.883551603011
shopping  done, size:  (150, 11)  max_dist =  4093.2469112320973
shopping  done, size:  (200, 11)  max_dist =  3926.908628146358
(2000, 11)  dropped to  (1395, 11)


In [56]:
dfseattle = df3.append([seattleextra1,seattleextra2,seattleextra3,seattleextra4,seattleextra5,
                        seattleextra6],ignore_index = True, sort=True)
dfseattle.shape

(22663, 11)

In [57]:
dropmask = dfseattle[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dfseattle = dfseattle[~dropmask]
dfseattle.shape

(17028, 11)

In [58]:
dfseattle.to_csv('SeattleBusinesses.csv', sep=';', compression='gzip')

### Extracting All Businesses for Charlotte and Pittsburgh, to check against Kaggle Dataset

In [6]:
maxradius = 7000
testlat = 40.445503
testlong = -79.995499 # Pittsburgh, PA

pittsburgh1 = AreaData('Pittsburgh1', coords=(testlat, testlong), radius=maxradius)
df4 = pittsburgh1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (250, 12)  max_dist =  7750.525108898224
arts  done, size:  (250, 12)  max_dist =  7909.538727200616
automotive  done, size:  (350, 12)  max_dist =  7521.3497536674895
beautysvc  done, size:  (400, 12)  max_dist =  7763.55307561897
education  done, size:  (150, 12)  max_dist =  14185.77910187415
eventservices  done, size:  (300, 12)  max_dist =  8845.67774322407
financialservices  done, size:  (150, 12)  max_dist =  8295.042186071585
food  done, size:  (700, 12)  max_dist =  7117.95758729452
health  done, size:  (500, 12)  max_dist =  7499.947088432784
homeservices  done, size:  (550, 12)  max_dist =  7505.20059626706
hotelstravel  done, size:  (150, 12)  max_dist =  8032.232496919575
localservices  done, size:  (400, 12)  max_dist =  7100.5965019308005
massmedia  done, size:  (31, 12)  max_dist =  0
nightlife  done, size:  (400, 12)  max_dist =  7060.846004200775
pets  done, size:  (100, 12)  max_dist =  15904.675712186943
professional  done, size:  (200, 12)  max

In [7]:
pittsburghextra1 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (150, 12)  max_dist =  3212.8273734255426
restaurants  done, size:  (100, 12)  max_dist =  2865.028072146682
restaurants  done, size:  (50, 12)  max_dist =  2922.4815207656848
restaurants  done, size:  (50, 12)  max_dist =  3597.2459352319534
restaurants  done, size:  (57, 12)  max_dist =  15519.364476306968
restaurants  done, size:  (450, 12)  max_dist =  2902.773640648724
restaurants  done, size:  (50, 12)  max_dist =  3129.1194350773085
restaurants  done, size:  (100, 12)  max_dist =  3128.86600540176
(1007, 12)  dropped to  (909, 12)


In [8]:
dfpittsburgh = df4.append([pittsburghextra1],ignore_index = True, sort=True)
dfpittsburgh.shape

(8040, 12)

In [9]:
dropmask = dfpittsburgh[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dfpittsburgh = dfpittsburgh[~dropmask]
dfpittsburgh.shape

(6044, 12)

In [10]:
dfpittsburgh.to_csv('PittsburghBusinessesUpdated.csv', sep=';', compression='gzip')

In [11]:
maxradius = 7000
testlat = 35.225327
testlong = -80.840803 # for Charlotte, NC

charlotte1 = AreaData('Charlotte1', coords=(testlat, testlong), radius=maxradius)
df5 = charlotte1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (250, 12)  max_dist =  7753.02415386466
arts  done, size:  (200, 12)  max_dist =  9117.238756573106
automotive  done, size:  (350, 12)  max_dist =  7026.41434340397
beautysvc  done, size:  (500, 12)  max_dist =  7617.108027729527
education  done, size:  (100, 12)  max_dist =  9035.54586654041
eventservices  done, size:  (300, 12)  max_dist =  7647.674184396606
financialservices  done, size:  (150, 12)  max_dist =  9040.214647461686
food  done, size:  (550, 12)  max_dist =  7754.754881283028
health  done, size:  (500, 12)  max_dist =  7552.762930243517
homeservices  done, size:  (700, 12)  max_dist =  7677.323951375351
hotelstravel  done, size:  (150, 12)  max_dist =  7079.663803269741
localservices  done, size:  (400, 12)  max_dist =  7264.503395927772
massmedia  done, size:  (30, 12)  max_dist =  12907.377242281269
nightlife  done, size:  (300, 12)  max_dist =  8248.619853653392
pets  done, size:  (100, 12)  max_dist =  8632.816141239216
professional  done, size: 

In [12]:
df5.shape

(6630, 12)

In [13]:
df5.to_csv('CharlotteBusinessesUpdated.csv', sep=';', compression='gzip')

### Extracting All Businesses for Phoenix and Las Vegas, to check against Kaggle Dataset

In [14]:
maxradius = 9000
testlat = 33.444506
testlong = -112.043629 # Phoenix, Arizona

Phoenix1 = AreaData('Phoenix1', coords=(testlat, testlong), radius=maxradius)
df6 = Phoenix1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (300, 12)  max_dist =  9216.173911442069
arts  done, size:  (300, 12)  max_dist =  9280.479190359581
automotive  done, size:  (950, 12)  max_dist =  9320.114813605429
beautysvc  done, size:  (650, 12)  max_dist =  9274.86951828378
education  done, size:  (250, 12)  max_dist =  9606.901625340657
eventservices  done, size:  (550, 12)  max_dist =  9763.218988785356
financialservices  done, size:  (350, 12)  max_dist =  9256.569529330875
food  done, size:  (800, 12)  max_dist =  9296.197390454472
health  done, size:  (1000, 12)  max_dist =  8132.3243723008545
homeservices  done, size:  (1000, 12)  max_dist =  6339.497687141464
hotelstravel  done, size:  (400, 12)  max_dist =  10085.46579728487
localservices  done, size:  (1000, 12)  max_dist =  8900.974245107165
massmedia  done, size:  (49, 12)  max_dist =  0
nightlife  done, size:  (300, 12)  max_dist =  9593.612808646656
pets  done, size:  (150, 12)  max_dist =  9634.6617374445
professional  done, size:  (850, 12)  m

In [16]:
# need extra for homeservices, health, restaurants
phoenixextra1 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (100, 12)  max_dist =  5363.330938186302
restaurants  done, size:  (300, 12)  max_dist =  4928.513424614224
restaurants  done, size:  (350, 12)  max_dist =  5173.47512315588
restaurants  done, size:  (400, 12)  max_dist =  5021.711219938967
restaurants  done, size:  (300, 12)  max_dist =  4947.2034279832005
restaurants  done, size:  (500, 12)  max_dist =  5084.74634218148
restaurants  done, size:  (300, 12)  max_dist =  5127.131114836009
restaurants  done, size:  (100, 12)  max_dist =  4939.0399444038285
(2350, 12)  dropped to  (2108, 12)


In [17]:
phoenixextra2 = yelp_extra_square_search('health', testlat, testlong)

health  done, size:  (50, 12)  max_dist =  5498.953215663654
health  done, size:  (100, 12)  max_dist =  4950.32693695319
health  done, size:  (300, 12)  max_dist =  5040.500883826042
health  done, size:  (650, 12)  max_dist =  4945.680771885298
health  done, size:  (350, 12)  max_dist =  4998.276828893452
health  done, size:  (200, 12)  max_dist =  5307.329295795806
health  done, size:  (250, 12)  max_dist =  5053.56506618935
health  done, size:  (50, 12)  max_dist =  5567.093253976082
(1950, 12)  dropped to  (1729, 12)


In [18]:
phoenixextra3 = yelp_extra_square_search('homeservices', testlat, testlong)

homeservices  done, size:  (100, 12)  max_dist =  7118.563662853862
homeservices  done, size:  (600, 12)  max_dist =  5188.104082099212
homeservices  done, size:  (500, 12)  max_dist =  5069.53492824791
homeservices  done, size:  (700, 12)  max_dist =  5199.775033763624
homeservices  done, size:  (450, 12)  max_dist =  5089.655778655415
homeservices  done, size:  (550, 12)  max_dist =  4997.116947701058
homeservices  done, size:  (500, 12)  max_dist =  5296.322148888558
homeservices  done, size:  (100, 12)  max_dist =  4981.501367764661
(3500, 12)  dropped to  (3098, 12)


In [19]:
phoenixextra4 = yelp_extra_square_search('shopping', testlat, testlong)

shopping  done, size:  (50, 12)  max_dist =  5044.541919381575
shopping  done, size:  (250, 12)  max_dist =  5112.456138304073
shopping  done, size:  (400, 12)  max_dist =  5220.612707991707
shopping  done, size:  (550, 12)  max_dist =  5269.123818721475
shopping  done, size:  (500, 12)  max_dist =  4989.926843003863
shopping  done, size:  (300, 12)  max_dist =  4997.116947701058
shopping  done, size:  (400, 12)  max_dist =  5296.322148888558
shopping  done, size:  (100, 12)  max_dist =  5715.155545531461
(2550, 12)  dropped to  (2257, 12)


In [20]:
dfphoenix = df6.append([phoenixextra1,phoenixextra2,phoenixextra3,phoenixextra4],
                       ignore_index = True, sort=True)
dfphoenix.shape

(20877, 12)

In [21]:
dropmask = dfphoenix[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dfphoenix = dfphoenix[~dropmask]
dfphoenix.shape

(15386, 12)

In [22]:
dfphoenix.to_csv('PhoenixBusinessesUpdated.csv', sep=';', compression='gzip')

In [23]:
dfphoenix.shape

(15386, 12)

In [25]:
maxradius = 7000
testlat = 36.120743
testlong = -115.192017 # Las Vegas dataset

lasvegas1 = AreaData('LasVegas1', coords=(testlat, testlong), radius=maxradius)
df7 = lasvegas1.extract_yelp_businesses(coords=(testlat, testlong))

active  done, size:  (324, 12)  max_dist =  38890.29232461535
arts  done, size:  (688, 12)  max_dist =  24078.80333810712
automotive  done, size:  (800, 12)  max_dist =  9373.449718604465
beautysvc  done, size:  (800, 12)  max_dist =  7187.21640849697
education  done, size:  (127, 12)  max_dist =  21295.23934318005
eventservices  done, size:  (650, 12)  max_dist =  7892.55983802774
financialservices  done, size:  (242, 12)  max_dist =  15252.910371347161
food  done, size:  (750, 12)  max_dist =  7580.555049266826
health  done, size:  (749, 12)  max_dist =  24078.80333810712
homeservices  done, size:  (1000, 12)  max_dist =  4683.620436891096
hotelstravel  done, size:  (550, 12)  max_dist =  9724.979845225467
localservices  done, size:  (750, 12)  max_dist =  8182.662346896977
massmedia  done, size:  (33, 12)  max_dist =  9421.156642015836
nightlife  done, size:  (708, 12)  max_dist =  14425.725751161803
pets  done, size:  (100, 12)  max_dist =  21303.428328325634
professional  done, si

In [27]:
# need extra for homeservices, restaurants, shopping
lasvegasextra1 = yelp_extra_square_search('restaurants', testlat, testlong)

restaurants  done, size:  (200, 12)  max_dist =  3665.4587952702336
restaurants  done, size:  (300, 12)  max_dist =  3528.0655858678333
restaurants  done, size:  (250, 12)  max_dist =  3842.100564692331
restaurants  done, size:  (200, 12)  max_dist =  3580.5415145043585
restaurants  done, size:  (350, 12)  max_dist =  3637.260451941886
restaurants  done, size:  (800, 12)  max_dist =  3513.5570091335508
restaurants  done, size:  (250, 12)  max_dist =  3794.6555040082385
restaurants  done, size:  (200, 12)  max_dist =  3655.2929829413133
(2550, 12)  dropped to  (2353, 12)


In [28]:
lasvegasextra2 = yelp_extra_square_search('homeservices', testlat, testlong)

homeservices  done, size:  (300, 12)  max_dist =  3654.348996274791
homeservices  done, size:  (400, 12)  max_dist =  3708.890378892827
homeservices  done, size:  (300, 12)  max_dist =  3647.354316991241
homeservices  done, size:  (200, 12)  max_dist =  3506.110877524267
homeservices  done, size:  (450, 12)  max_dist =  3579.126463281724
homeservices  done, size:  (300, 12)  max_dist =  3636.772523907286
homeservices  done, size:  (300, 12)  max_dist =  3640.7510454410844
homeservices  done, size:  (450, 12)  max_dist =  3692.089487255658
(2700, 12)  dropped to  (2459, 12)


In [29]:
lasvegasextra3 = yelp_extra_square_search('shopping', testlat, testlong)

shopping  done, size:  (200, 12)  max_dist =  4377.997711941034
shopping  done, size:  (250, 12)  max_dist =  3632.083675958216
shopping  done, size:  (250, 12)  max_dist =  3575.828481562725
shopping  done, size:  (450, 12)  max_dist =  3681.887058606982
shopping  done, size:  (500, 12)  max_dist =  3605.137450012219
shopping  done, size:  (900, 12)  max_dist =  3511.6347180558723
shopping  done, size:  (350, 12)  max_dist =  3651.973805114567
shopping  done, size:  (450, 12)  max_dist =  3565.729595856014
(3350, 12)  dropped to  (2864, 12)


In [30]:
dflasvegas = df7.append([lasvegasextra1,lasvegasextra2,lasvegasextra3],
                       ignore_index = True, sort=True)
dflasvegas.shape

(18970, 12)

In [31]:
dropmask = dflasvegas[['alias','latitude','longitude','review_count']].duplicated(keep='first')
dflasvegas = dflasvegas[~dropmask]
dflasvegas.shape

(14880, 12)

In [32]:
dflasvegas.to_csv('LasVegasBusinessesUpdated.csv', sep=';', compression='gzip')

### Extracting Business Categories with 1000+ Results Within a City

In [ ]:
# searching by sub-category does not work. Almost all businesses in a main category aren't also tagged
# with any of that category's subcategories
allcats = pd.read_json('Yelp_categoriesList.json')

In [62]:
allcats.head()

,alias,country_blacklist,country_whitelist,parents,title
0,3dprinting,NaN,NaN,[localservices],3D Printing
1,abruzzese,NaN,[IT],[italian],Abruzzese
2,absinthebars,NaN,[CZ],[bars],Absinthe Bars
3,acaibowls,"[TR, PL, AR, MX, IT, CL]",NaN,[food],Acai Bowls
4,accessories,NaN,NaN,[fashion],Accessories


In [70]:
allcats[allcats['parents'].str.contains('food',na=False, regex=False)]

,alias,country_blacklist,country_whitelist,parents,title
3,acaibowls,"[TR, PL, AR, MX, IT, CL]",NaN,[food],Acai Bowls
107,backshop,NaN,"[CH, AT, DE]",[food],Backshop
110,bagels,"[ES, AU]",NaN,[food],Bagels
113,bakeries,NaN,NaN,[food],Bakeries
138,beer_and_wine,NaN,NaN,[food],"Beer, Wine & Spirits"
148,bento,NaN,[JP],[food],Bento
152,beverage_stores,NaN,"[NL, PT, DE, CH, AT, BE, US, AU, ES, CL, TR, IT]",[food],Beverage Store
198,breweries,NaN,NaN,[food],Breweries
204,bubbletea,"[TR, CH, AR, MX, AT, ES]",NaN,[food],Bubble Tea
220,butcher,"[CH, DE, AT]",NaN,[food],Butcher


### Function test cases

In [32]:
testlocation3 = '125 Summer Street, Boston, MA'
test3 = SearchData('test3', 'education', location=testlocation3)
test3.add50rows()

50 rows added to test3


768.4505496780738

In [29]:
test3.printparams()

125 Summer Street, Boston, MA = location,  50 = limit,  0 = offset,  1000 = radius,  education = main_category
Dataframe size: (0, 1)


In [25]:
search_results = yelp_api.search_query(location=testlocation3, limit=50, offset=0,
                                               categories='education')
search_results

{'businesses': [{'id': '_1DNBfWm57A9pnhiB6-qsg',
   'alias': 'boston-photography-workshops-boston',
   'name': 'Boston Photography Workshops',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Pn49T2W22ipR3mNp2cNuDQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/boston-photography-workshops-boston?adjust_creative=Lt6caCYWdiuAzrXR2HVsuA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Lt6caCYWdiuAzrXR2HVsuA',
   'review_count': 56,
   'categories': [{'alias': 'photoclasses', 'title': 'Photography Classes'}],
   'rating': 5.0,
   'coordinates': {'latitude': 42.3456546, 'longitude': -71.0511046},
   'transactions': [],
   'location': {'address1': '15 Channel Center St',
    'address2': 'Ste 102',
    'address3': '',
    'city': 'Boston',
    'zip_code': '02210',
    'country': 'US',
    'state': 'MA',
    'display_address': ['15 Channel Center St',
     'Ste 102',
     'Boston, MA 02210']},
   'phone': '+16172215361',
   'display_phone': '(6

In [13]:
test3.df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
main_category    0 non-null object
dtypes: object(1)
memory usage: 0.0+ bytes


In [62]:
test3.df[test3.df['price'].isnull()]

,alias,categories,coordinates,distance,price,rating,review_count,url
4,martins-news-shops-boston-6,"[{'alias': 'mags', 'title': 'Newspapers & Maga...","{'latitude': 42.3523280591267, 'longitude': -7...",119.884831,NaN,1.0,3.0,https://www.yelp.com/biz/martins-news-shops-bo...
6,studio-verticale-boston,"[{'alias': 'homedecor', 'title': 'Home Decor'}...","{'latitude': 42.3524858, 'longitude': -71.0593...",161.854397,NaN,5.0,2.0,https://www.yelp.com/biz/studio-verticale-bost...
10,marquis-leathers-boston,"[{'alias': 'leather', 'title': 'Leather Goods'}]","{'latitude': 42.3512435, 'longitude': -71.0571...",192.235456,NaN,3.0,1.0,https://www.yelp.com/biz/marquis-leathers-bost...
13,sheehans-church-goods-company-boston,"[{'alias': 'religiousitems', 'title': 'Religio...","{'latitude': 42.3539123535156, 'longitude': -7...",204.499002,NaN,5.0,1.0,https://www.yelp.com/biz/sheehans-church-goods...
15,kabloom-boston-2,"[{'alias': 'florists', 'title': 'Florists'}]","{'latitude': 42.3510953038931, 'longitude': -7...",223.020636,NaN,5.0,1.0,https://www.yelp.com/biz/kabloom-boston-2?adju...
19,tangs-furniture-boston-2,"[{'alias': 'furniture', 'title': 'Furniture St...","{'latitude': 42.35093, 'longitude': -71.05817}",225.837320,NaN,1.0,7.0,https://www.yelp.com/biz/tangs-furniture-bosto...
25,campus-camera-and-electronics-boston,"[{'alias': 'photographystores', 'title': 'Phot...","{'latitude': 42.35067, 'longitude': -71.05681}",259.758424,NaN,1.0,1.0,https://www.yelp.com/biz/campus-camera-and-ele...
27,sulgrave-news-boston,"[{'alias': 'media', 'title': 'Books, Mags, Mus...","{'latitude': 42.35527, 'longitude': -71.05766}",261.694182,NaN,5.0,1.0,https://www.yelp.com/biz/sulgrave-news-boston?...
29,pshycic-readings-by-stacy-boston,"[{'alias': 'cosmetics', 'title': 'Cosmetics & ...","{'latitude': 42.3504295, 'longitude': -71.0577...",275.778178,NaN,1.0,1.0,https://www.yelp.com/biz/pshycic-readings-by-s...
31,oriental-fortune-giftland-boston,"[{'alias': 'giftshops', 'title': 'Gift Shops'}]","{'latitude': 42.3531312, 'longitude': -71.0608...",284.285236,NaN,3.5,2.0,https://www.yelp.com/biz/oriental-fortune-gift...


In [65]:
test3.df[~test3.df['price'].isnull()]['review_count'].mean()

26.276752767527675

In [66]:
test3.df[~test3.df['price'].isnull()]['price'].value_counts()

$$      150
$$$      65
$        31
$$$$     25
Name: price, dtype: int64

In [49]:
testlocation = "117 Vine Street Lexington MA"
test2 = SearchData('test2', testlocation, 'shopping')
test2.printparams()

117 Vine Street Lexington MA = location,  50 = limit,  0 = offset,  1000 = radius,  shopping = main_category
Dataframe size: (0, 0)


In [50]:
test2.addyelpdata()

50 rows added to test2


In [52]:
test2.df

,alias,categories,coordinates,distance,price,rating,url
0,emilia-creations-lexington,"[{'alias': 'bridal', 'title': 'Bridal'}, {'ali...","{'latitude': 42.4463804, 'longitude': -71.2247...",753.646467,$$$,5.0,https://www.yelp.com/biz/emilia-creations-lexi...
1,big-picture-framing-lexington,"[{'alias': 'framing', 'title': 'Framing'}]","{'latitude': 42.4466054, 'longitude': -71.225207}",779.923995,NaN,5.0,https://www.yelp.com/biz/big-picture-framing-l...
2,hitrons-lexington,"[{'alias': 'mattresses', 'title': 'Mattresses'...","{'latitude': 42.44632, 'longitude': -71.22583}",822.302169,$$$,5.0,https://www.yelp.com/biz/hitrons-lexington?adj...
3,ride-studio-cafe-lexington,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 42.4468886050926, 'longitude': -7...",859.454554,$$,4.0,https://www.yelp.com/biz/ride-studio-cafe-lexi...
4,two-aprons-cookery-lexington-2,"[{'alias': 'cookingclasses', 'title': 'Cooking...","{'latitude': 42.454226, 'longitude': -71.221343}",874.291357,NaN,5.0,https://www.yelp.com/biz/two-aprons-cookery-le...
5,shemin-nursuries-lexington,"[{'alias': 'gardening', 'title': 'Nurseries & ...","{'latitude': 42.4400101, 'longitude': -71.2115...",895.937868,$$$$,2.5,https://www.yelp.com/biz/shemin-nursuries-lexi...
6,seasons-four-lexington,"[{'alias': 'furniture', 'title': 'Furniture St...","{'latitude': 42.440016, 'longitude': -71.211575}",895.937868,$$$,3.5,https://www.yelp.com/biz/seasons-four-lexingto...
7,cvs-pharmacy-lexington-24,"[{'alias': 'drugstores', 'title': 'Drugstores'}]","{'latitude': 42.4473596, 'longitude': -71.2268...",909.370867,$$,2.5,https://www.yelp.com/biz/cvs-pharmacy-lexingto...
8,artinian-jewelry-lexington-2,"[{'alias': 'jewelry', 'title': 'Jewelry'}]","{'latitude': 42.4470482, 'longitude': -71.22715}",926.108604,$$,4.5,https://www.yelp.com/biz/artinian-jewelry-lexi...
9,greater-boston-running-company-lexington-lexin...,"[{'alias': 'sportswear', 'title': 'Sports Wear...","{'latitude': 42.447065, 'longitude': -71.2268863}",931.741693,$$,4.0,https://www.yelp.com/biz/greater-boston-runnin...


In [38]:
testlocation = "117 Vine Street Lexington MA"
test1 = SearchData('test1', testlocation, 'shopping')
test1.printparams()

117 Vine Street Lexington MA = location,  50 = limit,  0 = offset,  1000 = radius,  shopping = main_category
Dataframe size: (0, 0)


In [39]:
test1.addyelpdata()

50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1
50 rows added to test1


In [43]:
test1.df[test1.df['price'].isnull()]

,alias,categories,coordinates,distance,price,rating,url
39,fine-fit-fashion-and-tailoring-burlington,"[{'alias': 'sewingalterations', 'title': 'Sewi...","{'latitude': 42.4972453, 'longitude': -71.1948...",5790.317382,NaN,4.5,https://www.yelp.com/biz/fine-fit-fashion-and-...
101,le-vision-woburn,"[{'alias': 'optometrists', 'title': 'Optometri...","{'latitude': 42.468953, 'longitude': -71.177214}",3961.367579,NaN,4.5,https://www.yelp.com/biz/le-vision-woburn?adju...
106,henna-cafe-newton,"[{'alias': 'hennaartists', 'title': 'Henna Art...","{'latitude': 42.3401794238955, 'longitude': -7...",11967.093929,NaN,5.0,https://www.yelp.com/biz/henna-cafe-newton?adj...
128,tully-ink-boston-2,"[{'alias': 'screen_printing_tshirt_printing', ...","{'latitude': 42.3544129, 'longitude': -71.1328...",12392.853831,NaN,5.0,https://www.yelp.com/biz/tully-ink-boston-2?ad...
160,rare-moving-and-trucking-boston,"[{'alias': 'movers', 'title': 'Movers'}, {'ali...","{'latitude': 42.28383, 'longitude': -71.06801}",21875.205891,NaN,4.5,https://www.yelp.com/biz/rare-moving-and-truck...
180,frameworks-burlington-3,"[{'alias': 'framing', 'title': 'Framing'}]","{'latitude': 42.48541, 'longitude': -71.18828}",4757.376221,NaN,5.0,https://www.yelp.com/biz/frameworks-burlington...
231,metropolitan-furniture-burlington,"[{'alias': 'interiordesign', 'title': 'Interio...","{'latitude': 42.472021, 'longitude': -71.210831}",2754.870924,NaN,4.5,https://www.yelp.com/biz/metropolitan-furnitur...
235,five-stars-movers-brighton-5,"[{'alias': 'movers', 'title': 'Movers'}, {'ali...","{'latitude': 42.351519, 'longitude': -71.174146}",11208.575068,NaN,3.5,https://www.yelp.com/biz/five-stars-movers-bri...
240,locks-and-keys-woburn-6,"[{'alias': 'locksmiths', 'title': 'Keys & Lock...","{'latitude': 42.504966, 'longitude': -71.13391}",9267.828149,NaN,3.5,https://www.yelp.com/biz/locks-and-keys-woburn...
243,adams-fireplace-shop-cambridge,"[{'alias': 'lighting', 'title': 'Lighting Fixt...","{'latitude': 42.3872926, 'longitude': -71.1415...",9051.435927,NaN,5.0,https://www.yelp.com/biz/adams-fireplace-shop-...


In [47]:
test1.df[['alias']].duplicated().sum()

0

In [20]:
test = {tupl[0] : tupl[1] for tupl in search_results['businesses'][0].items() if tupl[0] in features}
test

{'alias': 'ride-studio-cafe-lexington',
 'url': 'https://www.yelp.com/biz/ride-studio-cafe-lexington?adjust_creative=Lt6caCYWdiuAzrXR2HVsuA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Lt6caCYWdiuAzrXR2HVsuA',
 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
  {'alias': 'bikes', 'title': 'Bikes'},
  {'alias': 'bike_repair_maintenance', 'title': 'Bike Repair/Maintenance'}],
 'rating': 4.0,
 'coordinates': {'latitude': 42.4468886050926, 'longitude': -71.2262222272653},
 'price': '$$',
 'distance': 859.4545537437085}